In [1]:
from functools import total_ordering, reduce
from collections import Counter
from itertools import combinations
import random

In [2]:
@total_ordering
class Card:
    def __init__(self, val_, suits_):
        
        assert(2 <= val_ & val_ <= 14)
        assert(suits_ in ['Spd', 'Hrt', 'Dmd', 'Clb'])
        
        self.val = int(val_)
        self.suits = suits_
        
    def __eq__(self, other):
        return self.val == other.val
    def __lt__(self, other):
        return self.val < other.val
    
    def __repr__(self):
        mapping = {11: 'J', 12: 'Q', 13: 'K', 14: 'A'}
        return '%s-%s'%(self.suits, self.val if self.val not in mapping.keys() else mapping[self.val])
    
    def __hash__(self):
        return hash((self.val, self.suits))

In [3]:
class Hand:
    
    def __init__(self, cards_=[]):
        assert(len(cards_) <= 2)
        self.cards = cards_
        
    def curent_hand(self):
        return Two_Cards(self.cards[0], self.cards[1])
        
    def best_hand_with_deck(self, deck):
        assert(len(deck) in [3, 4, 5])
        all_cards = set(self.cards).union(deck)
        
        return max([Five_Cards(any_five) for any_five in combinations(all_cards, 5)])
    
    def make_combo(self, deck=[]):
        if not deck:
            return self.curent_hand()
        else:
            return self.best_hand_with_deck(deck)
    
    def __repr__(self):
        return str(sorted(self.cards, reverse=True))

In [4]:
@total_ordering
class Two_Cards:
    
    level_mapping = {
        2: 'Pair',
        1: 'High Card'
    }
    
    def __init__(self, c1_, c2_):
        self.c1 = max(c1_, c2_)
        self.c2 = min(c1_, c2_)
        
        if self.c1.val == self.c2.val:
            self.level = 2
        else:
            self.level = 1
    
    def __eq__(self, other):
        return self.c1 == other.c1 and self.c2 == other.c2
    
    def __lt__(self, other):
        
        if self.level < other.level:
            return True
        
        elif self.level > other.level:
            return False
        
        elif self.level == 2: # [PAIR]
            return self.c1 < other.c1
        
        elif self.level == 1: # [NONE]
            if self.c1 < other.c1:
                return True
            elif self.c1 == other.c1:
                return self.c2 < other.c2
            else:
                return False
        else:
            raise exception('Fail to compare')
            
    def compare(self, other):
        '''1: win, 0: even, -1: lose'''
        print(self.level_mapping[self.level], '-', self.level_mapping[other.level])
        if self > other:
            return 1
        elif self < other:
            return -1
        elif self == other:
            return 0
    
    def __repr__(self):
        
        #if self.level == 2:
        #    return '[PAIR] %d-%d'%(self.c1.val, self.c2.val)
        #else:
        #    return '[NONE] %d-%d'%(self.c1.val, self.c2.val)
        return str([self.c1, self.c2])

In [5]:
@total_ordering
class Five_Cards:
    
    level_mapping = {
        9: 'Straight flush',
        8: 'Four of a kind',
        7: 'Full house',
        6: 'Flush',
        5: 'Straight', 
        4: 'Three of a kind',
        3: 'Two pair',
        2: 'Pair',
        1: 'High Card'
    }
    
    def __init__(self, cards_=[]):
        
        assert(len(cards_) == 5)
        self.cards = set(cards_)
        self.counter = Counter([card.val for card in self.cards])
        
        '''
        9. Straight flush (including Royal flush)
        8. Four of a kind (Done)
        7. Full house (Done)
        6. Flush (Done)
        5. Straight 
        4. Three of a kind (Done)
        3. Two pair (Done)
        2. Pair (Done)
        1. High Card   
        '''
        
        nums = sorted(self.counter.values())
        
        if nums == [1, 4]:
            self.level = 8
        elif nums == [2, 3]:
            self.level = 7
        elif nums == [1, 1, 3]:
            self.level = 4
        elif nums == [1, 2, 2]:
            self.level = 3
        elif nums == [1, 1, 1, 2]:
            self.level = 2
        else:
            FLUSH = self.__check_flush__(self.cards)
            STRAIGHT = self.__check_straight__(self.cards)
            
            if FLUSH & STRAIGHT:
                self.level = 9
            elif FLUSH:
                self.level = 6
            elif STRAIGHT:
                self.level = 5
            else:
                self.level = 1
    
    @staticmethod
    def __check_flush__(cards):
        return len(set([card.suits for card in cards])) == 1
    
    @staticmethod
    def __check_straight__(cards):
        return max(cards).val - min(cards).val == 4
    
    def sorted_nums(self, exclude=[]):
        return sorted([card.val for card in self.cards if card.val not in exclude], reverse=True)
    
    def __eq__(self, other):
        return False if self.level != other.level else self.sorted_nums() == other.sorted_nums()
    
    def __lt__(self, other):
        if self.level < other.level:
            return True
        
        elif self.level > other.level:
            return False
        
        # levels tie
        elif self.level in [8, 4, 2]:
            # four of a kind, three of a kind, one pair
            P_self = self.counter.most_common(1)[0][0]
            P_other = other.counter.most_common(1)[0][0]
            
            if P_self < P_other:
                return True
            elif P_self > P_other:
                return False
            else:
                return self.sorted_nums([P_self]) < other.sorted_nums([P_other])
        
        elif self.level == 7:
            # full house
            P_self = self.counter.most_common(1)[0][0]
            P_other = other.counter.most_common(1)[0][0]
            
            if P_self < P_other:
                return True
            elif P_self > P_other:
                return False
            else:
                return self.counter.most_common(2)[1][0] < other.counter.most_common(2)[1][0]
        
        elif self.level == 3:
            # two pairs
            P_self = sorted([c[0] for c in self.counter.most_common(2)], reverse=True)
            P_other = sorted([c[0] for c in other.counter.most_common(2)], reverse=True)
            
            if P_self < P_other:
                return True
            elif P_self > P_other:
                return False
            else:
                return self.sorted_nums(P_self) < other.sorted_nums(P_other)
            
        elif self.level in [9, 6, 5, 1]:
            # Straight flush, Flush, Straight, High Card
            return self.sorted_nums() < other.sorted_nums()
        
    def compare(self, other):
        '''1: win, 0: even, -1: lose'''
        print(self.level_mapping[self.level], '-', self.level_mapping[other.level])
        if self > other:
            return 1
        elif self < other:
            return -1
        elif self == other:
            return 0
        
    def __repr__(self):
        return str(sorted(self.cards, reverse=True))

In [6]:
class Pool:
    def __init__(self, exclude=[]):
        Q = reduce(lambda a, b: a + b, [[Card(i, suit) for i in range(2, 15)] for suit in ['Spd', 'Hrt', 'Dmd', 'Clb']])
        self.queue = [c for c in Q if c not in exclude]
        random.shuffle(self.queue)
        
    def draw(self, n=1):
        assert(n < len(self.queue))
        return [self.queue.pop() for i in range(n)]
    
    @property
    def n_remain(self):
        return len(self.queue)

In [7]:
def One_Game():

    P = Pool()

    player_1 = Hand(P.draw(2))
    player_2 = Hand(P.draw(2))
    
    Deck = []

    print('Player 1:', player_1)
    print('Player 2:', player_2)
    print('\t')

    for i, n_draws in enumerate([0, 3, 1, 1]):

        print('ROUND %d'%i)
        Deck.extend(P.draw(n_draws))

        print('Deck:', Deck)
        h1 = player_1.make_combo(Deck)
        h2 = player_2.make_combo(Deck)

        print(h1)
        print(h2)
        print('Result:', h1.compare(h2))
        print('\t')

        assert(len(set(player_1.cards + player_2.cards + Deck + P.queue)) == 52)

In [8]:
One_Game()

Player 1: [Hrt-7, Spd-3]
Player 2: [Spd-K, Spd-7]
	
ROUND 0
Deck: []
[Hrt-7, Spd-3]
[Spd-K, Spd-7]
High Card - High Card
Result: -1
	
ROUND 1
Deck: [Dmd-4, Dmd-Q, Dmd-7]
[Dmd-Q, Dmd-7, Hrt-7, Dmd-4, Spd-3]
[Spd-K, Dmd-Q, Dmd-7, Spd-7, Dmd-4]
Pair - Pair
Result: -1
	
ROUND 2
Deck: [Dmd-4, Dmd-Q, Dmd-7, Dmd-10]
[Dmd-Q, Dmd-10, Dmd-7, Hrt-7, Dmd-4]
[Spd-K, Dmd-Q, Dmd-10, Dmd-7, Spd-7]
Pair - Pair
Result: -1
	
ROUND 3
Deck: [Dmd-4, Dmd-Q, Dmd-7, Dmd-10, Hrt-10]
[Dmd-Q, Hrt-10, Dmd-10, Dmd-7, Hrt-7]
[Spd-K, Hrt-10, Dmd-10, Dmd-7, Spd-7]
Two pair - Two pair
Result: -1
	
